In [69]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
import datetime 
import time


## Spotipy documentation: 

https://spotipy.readthedocs.io/en/master/#

In [70]:
client_id = 'bf76bf609f1a4be6b656c55e4d77abd4'
client_secret = '8949a39342a74eddb3405b7a4f747852'

In [71]:
redirect_uri = 'https://localhost:8888/callback/'
scope= "user-read-recently-played"

In [72]:
today = datetime.datetime.now()
# because everyday we want to see the songs we've listed to for the 
# previous 24 hrs
yesterday = today - datetime.timedelta(days =1)
# unix timestamp in miliseconds, that's why need to * 1000
yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

In [73]:
nextmonth = today + datetime.timedelta(days =30)
# unix timestamp in miliseconds, that's why need to * 1000
nextmonth_unix_timestamp = int(nextmonth.timestamp()) * 1000

## Following this stackoverflow to auto refresh the token 

https://stackoverflow.com/questions/48883731/refresh-token-spotipy#:~:text=So%20it%20will%20be%20refreshed,access%20token%20%2F%20refresh%20token%20previously.

In [83]:
def create_spotify():
    auth_manager=SpotifyOAuth(scope=scope,
                                client_id =client_id ,
                                client_secret = client_secret,
                                redirect_uri = redirect_uri)
    spotify = spotipy.Spotify(auth_manager=auth_manager,requests_timeout=30 )
    return auth_manager,spotify

In [84]:
def refresh_spotify(auth_manager, spotify):
    token_info = auth_manager.cache_handler.get_cached_token()
    if auth_manager.is_token_expired(token_info):
        auth_manager, spotify = create_spotify()
    return auth_manager, spotify

In [ ]:
auth_manager, spotify = create_spotify()


In [ ]:
auth_manager, spotify = refresh_spotify(auth_manager, spotify)


In [ ]:
token_info = auth_manager.cache_handler.get_cached_token()


In [90]:
data = spotify.current_user_recently_played(after =yesterday_unix_timestamp)


In [92]:
while True: 
    empty = []
    auth_manager, spotify = refresh_spotify(auth_manager, spotify)
    data = spotify.current_user_recently_played(after =yesterday_unix_timestamp)
    if data['items'] != []:
        empty.append(data['items'][0]['track'])
        break 
    else:
        print('Nothing is playing.')

    # time.sleep(30)

In [15]:
artist_name = []
song_names = []
played_at_list = []
timestamps = []

for i in data['items']:
    song_names.append(i['track']['name'])
    played_at_list.append(i['played_at'])
    timestamps.append(i['played_at'][:10])
    artist_name.append(i['track']['artists'][0]['name'])

song_df = pd.DataFrame([played_at_list,timestamps,artist_name,song_names]).T
song_df.columns = ['played_at_list','timestamps','artist_name','song_names']

In [16]:
song_df

,played_at_list,timestamps,artist_name,song_names
0,2022-10-15T01:00:51.534Z,2022-10-15,Oatmello,Community
1,2022-10-15T00:59:20.369Z,2022-10-15,Vincent Rayn,Garden
2,2022-10-15T00:57:45.142Z,2022-10-15,kyoshi,balcony
3,2022-10-15T00:55:49.691Z,2022-10-15,Moonspatz,A Man Who Believed in Dreams
4,2022-10-15T00:54:15.290Z,2022-10-15,Saiko,Second Chance
5,2022-10-15T00:52:22.560Z,2022-10-15,Oaty.,Concrete Jungle
6,2022-10-15T00:50:35.572Z,2022-10-15,Mike Gusavo,More Than This
7,2022-10-15T00:48:27.725Z,2022-10-15,Ramsey Road,Relax Still
8,2022-10-15T00:45:49.838Z,2022-10-15,SōNE,Halfway There
9,2022-10-15T00:43:56.325Z,2022-10-15,Sepalot,Dream Chaser
